In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt

In [37]:
def release_token(accessToken):
    server = 'api.nrgstream.com'
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
print(default_pickle['accessToken'])
#release_token(default_pickle['accessToken'])

OOLEuob8mZT9z-jXldaikn8-cXKMQdx1MGpCSSICjfjlZWFtSm_2OKhYyrH71TGkXHapY7iNrTKtVrmq1T3tf3dylDC0JyTMzNlDpswwSvY6U9yFqyMNoedOhNjDJJx0n2Uv3MUNHGd22f_Z13Bm7KYiZQUBt9-GFU1b9aDIeotKOfsizyt1r7NJMIAMGEgSfWXh8WiHwv1iPRIpv6ppxR9XHQtux4HxAFcn1G24QUzodkf3y4oNn2AVh4rtDMMFTRzyPF1oS3v7P5mL0ra8uM_pG71SCuDv2LiTNeQZMMEojAW0kR3jxJLJLjitT9tms2RnvpJQPo7zl0eVT8vm2vvoGvQ


In [19]:
t = datetime(2022,1,1).date()
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (datetime.today().date(),monthly)
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (datetime.today().date(),daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:

with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle['alert_dates']

{'Biomass & Other': datetime.date(2022, 1, 1),
 'Coal': datetime.date(2022, 1, 1),
 'Dual Fuel': datetime.date(2022, 1, 1),
 'Energy Storage': datetime.date(2022, 1, 1),
 'Hydro': datetime.date(2022, 1, 1),
 'Natural Gas': datetime.date(2022, 1, 1),
 'Solar': datetime.date(2022, 1, 1),
 'Wind': datetime.date(2022, 1, 1),
 'Intertie': datetime.date(2022, 1, 1)}

In [9]:
default_pickle

{'accessToken': '',
 'current_data': '',
 'daily_outage_dfs': [(datetime.date(2022, 9, 7),
        timeStamp  value  fuelType
   0   2022-09-02    290  Intertie
   1   2022-09-03      0  Intertie
   2   2022-09-04      0  Intertie
   3   2022-09-05      0  Intertie
   4   2022-09-06      0  Intertie
   ..         ...    ...       ...
   115 2022-12-26      0      Wind
   116 2022-12-27      0      Wind
   117 2022-12-28      0      Wind
   118 2022-12-29      0      Wind
   119 2022-12-30      0      Wind
   
   [1323 rows x 3 columns]),
  (datetime.date(2022, 9, 7),
        timeStamp  value  fuelType
   0   2022-09-02    290  Intertie
   1   2022-09-03      0  Intertie
   2   2022-09-04      0  Intertie
   3   2022-09-05      0  Intertie
   4   2022-09-06      0  Intertie
   ..         ...    ...       ...
   115 2022-12-26      0      Wind
   116 2022-12-27      0      Wind
   117 2022-12-28      0      Wind
   118 2022-12-29      0      Wind
   119 2022-12-30      0      Wind
   
  